In [ ]:
import json,csv,time
import pandas as pd
from urllib import error
from urllib.request import urlopen
from datetime import date, datetime, timedelta

In [ ]:
# API of Gemini order book, see https://docs.gemini.com/rest-api/#current-order-book
# It returns 100 lines 
api_url = "https://api.gemini.com/v1/book/btcusd"

# Get data every 1 mins(60s). 
timeout = 60

In [ ]:
# This function runs <=1s in my computer 
def get_data(): 
    check = True
    while check:
        try:
            network = json.loads(urlopen(api_url).read())
            flag = True
            check = False
            #print ('Network is OK.')
            pass
        except:
            print ("Network or server issue. wait 5 min to restart" )
            time.sleep(300)
            print ("Restart...")
    flag = True        
    #Get second start from zero
    while flag:
        bookdata = json.loads(urlopen(api_url).read())
        timestamp = int(bookdata["bids"][0]["timestamp"])
        remainder1 = timestamp%60
        if remainder1 < 1:
            flag = False
        else:
            #time.sleep(0.8)
            #print ('Waiting '+str(60 - remainder1%60)+'s to start next download.')
            bookdata2 = json.loads(urlopen(api_url).read())
            timestamp2 = int(bookdata["bids"][0]["timestamp"])
            remainder2 = timestamp2%60
            if remainder2 < remainder1:
                # This line may cause some deviation of time in seconds
                # but guranteens collecting data near 0s of this minute
                timestamp = timestamp-remainder1
                flag = False



    bids = bookdata["bids"]
    asks = bookdata["asks"]
    timestr = datetime.fromtimestamp(timestamp).strftime("%Y%m%d_%H%M%S")

    all_data_rw = bids+asks
    prices = []
    amounts = []
    for d in all_data_rw:
        prices.append(d["price"])
        amounts.append(d["amount"])

    df= pd.DataFrame(columns=["buy_indicator", "market_size", "price"])
    df["price"] = prices
    df["market_size"] = amounts

    df.loc[0:len(bids),("buy_indicator")] = 1
    df.loc[len(bids):, ("buy_indicator")] = 0
    #print (len(bids),len(asks))

    df.to_csv('bitcoin_'+timestr+'.txt', index=False, sep=' ', 
                header=["buy_indicator", "market_size", "price"])
    return timestamp

In [ ]:
# Execution
count = 0
while True :
    timestamp = get_data()
    count += 1
    print (str(count) + ' Data File has been downloaded!')
    sleept = timeout-(time.time() - timestamp+5)
    if sleept > 0:
        time.sleep(sleept)